<a href="https://colab.research.google.com/github/upsoprog/Tasks/blob/Big-data/Laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Laboratorio 1
---
#Análisis de Datos de Ventas Mensuales
---

## Introducción

En este informe se presenta un análisis de los datos de ventas de una tienda online durante el año 2019. El objetivo es identificar los patrones, las tendencias y los factores que influyen en el comportamiento de las ventas y proponer estrategias para optimizar el negocio. Para ello, se plantean las siguientes preguntas clave:

* ¿Cómo variaron las ventas a lo largo de los diferentes meses?
* ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la probabilidad de compra?
* ¿En qué ciudades se han registrado las mayores ventas?
* ¿Cuál es el producto más vendido en general y en cada mes?
* ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?
* ¿Se ha observado algún aumento o disminución significativa en las ventas en días cercanos a eventos especiales, como días festivos?

## Presentación de los datos

Este informe se basa en los datos de ventas de una tienda online correspondientes al año 2019. Los datos provienen de 12 archivos csv, cada uno con la información de los pedidos de un mes. Cada pedido tiene un ID único, un producto, una cantidad, un precio, una fecha y una dirección de envío.

La cantidad de pedidos varía de un mes a otro, siendo el mes con más ventas diciembre (25.117) y el mes con menos ventas enero (9.723). El total de pedidos en el año fue de 177.860. La tienda online ofrece 19 productos distintos, de los cuales el más vendido fue las baterías AAA (4-pack). El producto con el precio más alto fue el Macbook Pro Laptop, que cuesta 1.700 dólares.

Para analizar los datos numéricos, se descartaron los valores NaN que aparecían en las columnas de cantidad y precio. Estos valores indican que hay datos ausentes o incorrectos que podrían alterar los resultados. Al eliminarlos, se obtuvieron solo los datos que se pueden usar para hacer cálculos matemáticos como sumas, promedios, desviaciones estándar, etc.



## Análisis

In [ ]:
import pandas as pd
import os
import plotly.io as pio
pio.renderers.default = "colab"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Se transforman y limpian para una vision más clara.

In [ ]:
ruta_de_acceso = "/content/drive/MyDrive/BigData/Dataset_de_ventas2"

archivos = os.listdir(ruta_de_acceso)

dataframes ={}

date_format = '%m/%d/%y %H:%M'

for archivo in archivos:
  if archivo.endswith('.csv'): #Asegurarse de que se consideren archivos csv
    mes = archivo.split('_')[3].split('.')[0] #Obtener el nombre del mes sin la extension
    ruta_completa = os.path.join(ruta_de_acceso,archivo) #Ruta de acceso completa
    df = pd.read_csv(ruta_completa) #Cargar el csv en un Dataframe
    df_ventas = df.copy() #Hago copia

    #Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce') #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    #Elimino todos los Nan y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()

    #Transformo todos los valores numericos al tipo de dato correspondiente
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)

    dataframes[mes] = df_ventas

    dataframes[mes]['Fecha de Pedido'] = pd.to_datetime(dataframes[mes]['Fecha de Pedido'], format=date_format)

In [ ]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  return datos

### Se hace una copia del DataFrame original para no modificarlo

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

dataframes_copia = dataframes
for mes in meses:
  dataframe_mes = dataframes_copia[mes]
  dataframes_copia[mes] = extraer_datos(dataframe_mes)

### 1. Tendencias de  ventas mensuales

#### ¿Cómo variaron las ventas a lo largo de los diferentes meses? y ¿Hubo algún mes que se destacó en términos de ventas?


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"

totales_ventas = []

for mes in meses:

  total_ventas_mes = dataframes_copia[mes]['Cantidad Pedida'].sum()
  totales_ventas.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = totales_ventas,
        yaxis= 'y',
        name = 'Unidades Vendidas',
        marker = dict(color= totales_ventas, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
        text = totales_ventas,
        textposition = 'auto',
)
)

fig.update_layout(
    title = 'Unidades Vendidas por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Unidades Vendidas',side='left')
)

#Muestro el grafico
fig.show(renderer="colab")

Como se observa en el grafico de arriba, las ventas variaron según el mes, mostrando una tendencia creciente hacia el final del año, con la excepción del mes de abril, donde hubo un leve aumento. El mes con menos ventas fue enero, con un total de 10.868 pedidos, mientras que el mes con más ventas fue diciembre, con un total de 28.121 pedidos.

El mes que se destacó en términos de ventas fue diciembre, que tuvo un aumento del 41.97% respecto al mes anterior (noviembre). Este aumento se puede atribuir a la temporada navideña, que suele impulsar el consumo.



#### ¿Cuál es el ingreso total generado por mes?


In [ ]:
import plotly.graph_objects as go

totales_ventas = []

for mes in meses:
  total_ventas_mes = (dataframes_copia[mes]['Cantidad Pedida']*dataframes_copia[mes]['Precio Unitario']).sum().round(2)
  totales_ventas.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = totales_ventas,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= totales_ventas, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
        text = totales_ventas,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingreso total por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show(renderer="colab")

Así mismo, el ingreso total generado por mes se obtiene multiplicando la cantidad pedida por el precio unitario de cada pedido y sumando los resultados. El mes que generó más ingresos fue diciembre, con un total de 4.619.297 millones de dólares, mientras que el mes que generó menos ingresos fue enero, con un total de 1.815.335 millones de dólares. El gráfico muestra la evolución de los ingresos por mes. El aumento de las ventas y los ingresos en diciembre se puede explicar por la temporada navideña, que suele estimular el consumo.

### 2. Optimización de la publicidad y patrón de ventas por hora

#### ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la probabilidad de compra? y ¿Cómo cambian los patrones de ventas por hora a lo largo del año?


In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    Ventas_por_hora = dataframes_copia[mes].groupby('Hora')['Cantidad Pedida'].sum()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show(renderer="colab")

Para responder a esta pregunta, se analizó la frecuencia de los pedidos por hora del día, considerando todas las horas del año. Se encontró que el momento del día con más pedidos fue entre las 11:00 y las 13:00, seguido por el intervalo entre las 18:00 y las 20:00. Estos momentos coinciden con las horas previas y posteriores al almuerzo y la cena, respectivamente, lo que sugiere que los clientes aprovechan estos momentos para hacer compras online. Por lo tanto, se podría decir que estos son los momentos ideales para presentar la publicidad y aumentar la probabilidad de compra. El gráfico de líneas muestra la frecuencia de los pedidos por hora del día.

####  ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad en los distintos meses?


In [ ]:
fig = go.Figure()

horas_analizadas = [12,19]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    ventas_hora = dataframes_copia[mes][dataframes_copia[mes]['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(
      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width=2, shape='spline')
  ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
)

fig.show(renderer="colab")

Para examinar los patrones de ventas por hora y por mes, se seleccionaron dos horas clave del día: las 12 y las 19 horas. Se analizaron los datos de todos los meses del año y se observó que la frecuencia de los pedidos variaba poco entre ellos, excepto en dos casos. Se detectó un aumento de casi el 10% en las ventas de la tarde (19 horas) entre febrero y mayo, lo que podría estar relacionado con factores estacionales como el clima, las festividades o las vacaciones. Estos resultados sugieren que los clientes tienen preferencias distintas según la hora y el mes del año, lo que podría tener implicaciones para las estrategias de marketing y promoción.

### 3. Distribución de ventas por ubicación

#### ¿En qué ciudades se han registrado las mayores ventas?

In [ ]:
for mes in meses:
    columnas = dataframes_copia[mes]['Dirección de Envio'].str.split(', ', expand=True)
    columnas = columnas.rename(columns={0: 'Calle', 1: 'Ciudad', 2: 'PO BOX'})

    columnas[['Estado', 'Cod Postal']] = columnas['PO BOX'].str.split(' ',expand=True)
    dataframes_copia[mes] = pd.concat([dataframes_copia[mes],columnas],axis=True)

In [ ]:
# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
  ventas_por_ciudad = dataframes_copia[mes].groupby('Ciudad')['Cantidad Pedida'].sum()

# Reseteo el indice
ventas_por_ciudad = ventas_por_ciudad.reset_index(inplace=False)
ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,2224
1,Austin,1520
2,Boston,3029
3,Dallas,2300
4,Los Angeles,4373
5,New York City,3828
6,Portland,1859
7,San Francisco,6786
8,Seattle,2202


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show(renderer="colab")

Las ciudades que han registrado las mayores ventas son San Francisco, Los Angeles y New York City, con 6786, 4373 y 3828 pedidos respectivamente. Estas ciudades son las más pobladas y las más importantes económicamente de los Estados Unidos, lo que puede explicar su alto nivel de ventas.

Las ciudades que han registrado las menores ventas son Portland, Austin y Seattle, con 1859, 1520 y 2202 pedidos respectivamente. Estas ciudades tienen una población menor y un mercado más reducido que las anteriores, lo que puede influir en su bajo nivel de ventas.

Las ciudades que han registrado un nivel medio de ventas son Boston, Dallas y Atlanta, con 3029, 2300 y 2224 pedidos respectivamente. Estas ciudades tienen una población y un mercado intermedios entre las anteriores, lo que puede reflejar su nivel medio de ventas.

#### ¿Cómo se comparan las ventas en diferentes estados o regiones?


In [ ]:

# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
  ventas_por_estado = dataframes_copia[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado = ventas_por_estado.reset_index(inplace=False)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,11159
1,GA,2224
2,MA,3029
3,ME,352
4,NY,3828
5,OR,1507
6,TX,3820
7,WA,2202


In [ ]:

import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show(renderer="colab")

Las ventas en diferentes estados o regiones varían según el tamaño y la actividad económica de cada uno. El estado que tiene las ventas más altas es California (CA), con 11159 pedidos. Esto se debe a que California es el estado más poblado y el más rico de los Estados Unidos, con una gran diversidad de sectores e industrias. El estado que tiene las ventas más bajas es Maine (ME), con solo 352 pedidos. Esto se explica por que Maine es uno de los estados más pequeños y menos poblados de los Estados Unidos, con una economía basada principalmente en la agricultura y la pesca.

Entre los estados con ventas intermedias, se pueden destacar algunos casos. Por ejemplo, Nueva York (NY) tiene 3828 pedidos, lo que lo sitúa en el segundo lugar después de California. Esto se debe a que Nueva York es el centro financiero y cultural más importante de los Estados Unidos, con una gran concentración de población y negocios en la ciudad de Nueva York. Por otro lado, Texas (TX) tiene 3820 pedidos, lo que lo sitúa en el tercer lugar. Esto se debe a que Texas es el segundo estado más grande y más poblado de los Estados Unidos, con una economía diversificada que incluye el petróleo, la tecnología, la agricultura y el comercio. Otro caso interesante es el de Massachusetts (MA), que tiene 3029 pedidos, lo que lo sitúa en el cuarto lugar. Esto se debe a que Massachusetts es uno de los estados más innovadores y educados de los Estados Unidos, con un fuerte sector de biotecnología, educación superior y salud.

#### ¿Existe variación en las ventas por estado a lo largo de los meses?


In [ ]:
ventas_por_estado_mes = {}
estados = []

fig = go.Figure()

for estado in dataframes_copia['Enero']['Estado']:
  if estado not in estados:
    estados.append(estado)

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes_copia[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)


# Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name=f'{mes}',
  ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por mes por estado",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
)

fig.show(renderer="colab")

Las ventas en los diferentes estados conservan la misma relación con respecto a la variación en cuanto a cantidades pedidas se refiere, se observa que diciembre es para todos los involucrados el mes en el que se realizan la mayor cantidad de pedidos. En contraposición enero es el mes que registra la menor cantidad de consumos. Dado el análisis realizado podemos inferir que no es una cuestión particular del estado, sino que es una situación general para todos los participantes en la evaluación.

### 4. Análisis del producto más vendido

#### ¿Cuál es el producto más vendido en general y en cada mes?


In [ ]:
cantidades_por_producto = {}

# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
    producto_mas_vendido = dataframes_copia[mes].groupby('Producto')['Cantidad Pedida'].sum()
    producto_mas_vendido = producto_mas_vendido.reset_index(inplace=False)

    # Itero sobre cada fila del DataFrame para actualizar las cantidades vendidas
    for index, row in producto_mas_vendido.iterrows():
        producto = row['Producto']
        cantidad_pedida = row['Cantidad Pedida']

        # Si el producto ya existe en el diccionario, actualizo la cantidad, de lo contrario, se agrega
        if producto in cantidades_por_producto:
            cantidades_por_producto[producto] += cantidad_pedida
        else:
            cantidades_por_producto[producto] = cantidad_pedida

# Encuentro la cantidad máxima pedida
max_cantidad = max(cantidades_por_producto.values())

# Encuentro el producto con esa cantidad máxima
producto_mas_vendido = next(key for key, value in cantidades_por_producto.items() if value == max_cantidad)

# Imprimo el producto más vendido
print(f'El producto más vendido es: {producto_mas_vendido} con una cantidad pedida de {max_cantidad}')

El producto más vendido es: AAA Batteries (4-pack) con una cantidad pedida de 31017


In [ ]:
fig = go.Figure()

productos = list(cantidades_por_producto.keys())  # Lista de nombres de productos
cantidades = list(cantidades_por_producto.values())  # Lista de cantidades

# Agregar el gráfico de barras
fig.add_trace(go.Bar(
    x=productos,
    y=cantidades,
    name="Productos",
    marker=dict(color=cantidades, colorscale='RdYlGn', colorbar=dict(title='Total de ventas')),
))

# Configurar ejes y título
fig.update_layout(
    title="Cantidad vendida de productos",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas"),
)

# Mostrar el gráfico
fig.show(renderer="colab")

Para determinar el producto más vendido en general, necesitamos sumar la cantidad de ventas para cada producto y luego encontrar el producto con la cantidad de ventas más alta. En este caso, el producto más vendido en general es AAA Batteries (4-pack), con un total de 31017 ventas.


In [ ]:
# Obtener el producto más vendido de cada mes y la cantidad correspondiente
productos_mas_vendidos = {}
cantidades_mas_vendidas = {}

for mes in meses:
    producto_mas_vendido = dataframes_copia[mes].groupby('Producto')['Cantidad Pedida'].sum()
    producto_mas_vendido = producto_mas_vendido.reset_index(inplace=False)

    max_cantidad = producto_mas_vendido['Cantidad Pedida'].max()
    producto_mas_vendido = producto_mas_vendido[producto_mas_vendido['Cantidad Pedida'] == max_cantidad]['Producto'].iloc[0]

    productos_mas_vendidos[mes] = producto_mas_vendido
    cantidades_mas_vendidas[mes] = max_cantidad

# Crear el gráfico de barras
fig = go.Figure()

for mes, producto in productos_mas_vendidos.items():
    cantidad_vendida = cantidades_mas_vendidas[mes]
    fig.add_trace(go.Bar(
        x=[f"{producto}"],
        y=[cantidad_vendida],
        name=f"{mes}",
    ))

# Configurar ejes y título
fig.update_layout(
    title="Producto más vendido por mes",
    xaxis=dict(title="Productos"),
    yaxis=dict(title="Ventas"),
)
# Mostrar el gráfico
fig.show(renderer="colab")


Ahora, respondiendo al producto más vendido en cada mes siempre son las "AAA Batteries (4-pack)", seguido de claro el "AA Batteries (4-pack)" en segundo lugar y los cables de carga en tercer y cuarto lugar. Esto sugiere que los clientes tienen una alta demanda de baterías y cables de carga.

Es posible que esta tendencia se deba a varios factores, como el aumento del uso de dispositivos electrónicos y la necesidad de reemplazar las baterías con frecuencia. También es posible que los precios competitivos y la calidad de los productos hayan influido en las ventas.

### 5. Tendencia ventas

####  ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dias = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]

fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

fila = 1
columna = 0

dataframe_max = 0
for mes in meses:
    for dia in dias:
        dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum().max()
        if (dataframe_dia > dataframe_max):
            dataframe_max = dataframe_dia

for mes in meses:
  columna+=1
  if (columna == 5):
    fila+=1
    columna=1
  for dia in dias:
      dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
      dataframe_dia = dataframe_dia.reset_index(inplace=False)

      # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
      bar_trace1 = go.Bar(
          x=dias,
          y=dataframe_dia['Cantidad Pedida'],
          name=f'{mes}',
      )

      fig.update_xaxes(title_text="Dias", row=fila, col=columna)
      fig.update_yaxes(title_text="Cantidad", row=fila, col=columna)
      fig.update_yaxes(range=[0, dataframe_max], row=fila, col=columna)

  fig.add_trace(bar_trace1, row=fila, col=columna)


# Configuro los ejes y los títulos
fig.update_layout(
    title='Productos por dia por mes',
)

fig.show(renderer="colab")

Se podría ver que las ventas tienden a aumentar al principio, a mediados y al final de cada mes, lo que podría indicar que los clientes compran más cuando reciben sus ingresos o cuando se acercan las fechas límite. También se podría ver que las ventas tienden a disminuir en los días intermedios de cada mes, lo que podría sugerir que los clientes compran menos cuando tienen más gastos o cuando tienen menos urgencia.

Estas son algunas de las posibles explicaciones para la pequeña tendencia o el patrón en las ventas que se repita a lo largo de los días en los diferentes meses.

#### ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [ ]:
dias_semana=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

fila = 1
columna = 0

weekday_color = 'blue'
weekend_color = 'red'

max_mes = 0
for mes in meses:
  for dia in dias:
    dia_de_semana = dataframes_copia[mes].groupby('Fecha de Pedido')['Cantidad Pedida'].sum()
    dia_de_semana = dia_de_semana.reset_index(inplace=False)
    dia_de_semana['Dia de Semana'] = dia_de_semana['Fecha de Pedido'].dt.day_name()

  suma_dia_de_semana = dia_de_semana.groupby('Dia de Semana')['Cantidad Pedida'].sum()
  maximo_mes = suma_dia_de_semana.max()
  if (maximo_mes > max_mes):
    max_mes = maximo_mes

for mes in meses:
  columna+=1
  if (columna == 5):
    fila+=1
    columna=1
  for dia in dias:
    dia_de_semana = dataframes_copia[mes].groupby('Fecha de Pedido')['Cantidad Pedida'].sum()
    dia_de_semana = dia_de_semana.reset_index(inplace=False)
    dia_de_semana['Dia de Semana'] = dia_de_semana['Fecha de Pedido'].dt.day_name()

  suma_dia_de_semana = dia_de_semana.groupby('Dia de Semana')['Cantidad Pedida'].sum()


  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  bar_trace1 = go.Bar(
      x=dias_semana,
      y=suma_dia_de_semana,
      showlegend=False,
      marker=dict(
        color=[weekday_color if dia in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] else weekend_color for dia in dias_semana])
  )

  fig.update_xaxes(title_text="Dia de semana", row=fila, col=columna)
  fig.update_yaxes(title_text="Cantidad", row=fila, col=columna)
  fig.update_yaxes(range=[0, max_mes + 100], row=fila, col=columna)

  fig.add_trace(bar_trace1, row=fila, col=columna)



# Configuro los ejes y los títulos
fig.update_layout(
    title='Productos por dia de semana por mes',
)

fig.show(renderer="colab")


Para el analisis de esta pregunta dividiremos el año en dos semestres para tener un analisis más claro y poder relacionarlos.
En el primer semestre se observan varias cosas, los primeros meses del año son meses más tranquilos donde las ventas no se elevan mucho y establecen una base.

Luego desde Marzo hasta Junio sucede lo contrario, las ventas se elevan, los días más fuertes de Marzo son al comienzo de semana y a mediados, en Abril los martes se destacan. En Mayo las ventas tienden a subir los lunes y los viernes, cayendo en los demás días. En Junio, se observa un aumento de ventas los días miercoles y jueves sobrepasando así a los demás en resto de la semana.

Yendo al segundo semestre, al igual que el primero, los primeros tres meses son meses más tranquilo en cuanto a ventas, siendo Septiembre el mes con menos ventas entre ellos. Aun así no se pierde una pequeña tendencia de compra en los días martes y jueves, destacando Julio con más compras en los días Martes.
Octubre, Noviembre y Diciembre, son los meses con más ventas del año. Esto se puede deber a la cercania de las fechas festivas como se va a analizar en el proximo punto. Viendo el grafico podemos decir que el mes de Octubre las ventas tuivieron un aumento en toda la semana, pero sobresale el día viernes, sabado y domingo. En Noviembre la tendendia cambia, los días lunes y miercoles es donde las ventas aumentaron. Y por ultimo en Diciembre, es mes con mayores ventas que se le puede atribuir a Navidad y Año nuevo, las mayores ventas se realizaron los días martes, miercoles y sabados.

### 6.Impacto de eventos especiales en las ventas

● ¿Se ha observado algún aumento disminución significativa en las ventas en días cercanos a eventos especiales,como días festivos?

● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de las ventas y cómo se manifestó ese impacto?

Analizaremos el impacto de eventos especiales, como días festivos y otros días conmemorativos, en las ventas de nuestros productos. Para llevar a cabo este análisis, se recopiló y examinó los distintos dataset de cada mes. A continuación, presentamos los resultados de nuestro análisis.



#### Día de la Independencia - 4 de Julio

Informe de Análisis de Ventas durante el Día de la Independencia - 4 de Julio

Introducción:
El presente informe tiene como objetivo analizar el impacto de un evento festivo en las ventas, específicamente en el Día de la Independencia de los Estados Unidos, que se celebra el 4 de Julio. En este análisis, examinamos las ventas en los dos días anteriores y los dos días posteriores al evento festivo para evaluar cualquier tendencia o cambio significativo en el comportamiento de las ventas.

Metodología:
Para llevar a cabo este análisis, se recopilaron y analizaron datos de ventas de productos durante el período cercano al Día de la Independencia. Se consideraron dos días anteriores al evento festivo, el propio 4 de Julio y dos días posteriores para evaluar cualquier patrón evidente.

In [ ]:
dataframe_julio = dataframes_copia["Julio"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_julio= dataframe_julio.reset_index(inplace=False)
dataframe_julio

,Dia,Cantidad Pedida
0,1,539
1,2,519
2,3,519
3,4,506
4,5,516
5,6,519
6,7,511
7,8,538
8,9,541
9,10,530


Promedia de ventas del mes de Julio

In [ ]:
promedio_julio = dataframe_julio["Cantidad Pedida"].mean()
promedio_julio

518.3548387096774

Ventas realizadas el 4 de Julio - Día de la independencia

In [ ]:
dataframe_julio_fest = dataframes_copia["Julio"].loc[dataframes_copia["Julio"]["Dia"] ==4].groupby('Dia')['Cantidad Pedida'].sum().reset_index()
dataframe_julio_fest

,Dia,Cantidad Pedida
0,4,506


Calculo del desvio standar

In [ ]:
desviacion_estandar = dataframe_julio['Cantidad Pedida'].std()

print(f"La desviación estándar de 'Cantidad Pedida' es: {desviacion_estandar}")


La desviación estándar de 'Cantidad Pedida' es: 26.44308149856565


La mayoría de los valores en la columna 'Cantidad Pedida' están a una distancia de aproximadamente 26.44 unidades de la unidad en la que medimos 'Cantidad Pedida' de la media.



Top de los 5 dias con mas ventas en el mes de Julio


In [ ]:
top_5_ventas_julio = dataframe_julio.nlargest(5, 'Cantidad Pedida')
print(top_5_ventas_julio)

    Dia  Cantidad Pedida
14   15              573
25   26              557
17   18              554
21   22              546
8     9              541


In [ ]:
#Día de la Independencia (Independence Day) - 4 de julio

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Datos para el mes de julio
mes = "Julio"
dias = list(range(1, 32))  # Lista de días del mes

fig = make_subplots(rows=1, cols=1, subplot_titles=[f"Productos por día en {mes}"])

dataframe_dia_ind = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_dia_ind = dataframe_dia_ind.reset_index(inplace=False)

# Colores de las barras, con rojo para el día 4 de julio
colores = ['red' if dia == 4 else 'blue' for dia in dataframe_dia_ind['Dia']]

# Agregar un gráfico de barras para julio con colores personalizados
bar_trace1 = go.Bar(
    x=dataframe_dia_ind['Dia'],
    y=dataframe_dia_ind['Cantidad Pedida'],
    name=f'{mes}',
    marker=dict(color=colores)  # Establecer colores personalizados
)

fig.update_xaxes(title_text="Días")
fig.update_yaxes(title_text="Cantidad de ventas")
fig.update_yaxes(range=[0, dataframe_dia_ind['Cantidad Pedida'].max()])  # Ajustar el rango del eje Y

fig.add_trace(bar_trace1)

# Configurar el título
fig.update_layout(
    title=f'Productos por día en {mes}',
    width=800,  # Ancho en píxeles
)

fig.show(renderer="colab")



Resultados:
Durante el análisis de las ventas en el período alrededor del Día de la Independencia, se observaron los siguientes hallazgos clave:

Disminución de Ventas Precedentes al Evento:

En los dos días anteriores al 4 de Julio, se registró una disminución gradual en las ventas.
Esta disminución puede atribuirse a una posible anticipación de los consumidores hacia el evento festivo, lo que podría haber llevado a una reducción en la actividad de compra en esos días.
Impacto Significativo en el Día de la Independencia:

El 4 de Julio, el Día de la Independencia, se destacó por una disminución notable en las ventas en comparación con los días anteriores.
Esto indica que el evento festivo tuvo un impacto significativo en el comportamiento de las ventas, con una marcada caída en la actividad comercial.
Recuperación Posterior al Evento:

Después del Día de la Independencia, en los dos días posteriores, se observó una recuperación gradual en las ventas.
Los consumidores parecen haber vuelto a su actividad de compra habitual después de la celebración del evento.

Las ventas realizadas en el día de la independencia se encuentran debajo del promedio de ventas del mes de Julio

####Dia del trabajador primer lunes de Septiembre


Introducción:
El presente informe tiene como objetivo analizar el impacto de un evento festivo en las ventas, específicamente en el Día del Trabajador (Labor Day) en Estados Unidos, que se celebra el primer lunes de septiembre. En este análisis, se examinan las ventas en el Día del Trabajador y se comparan con los días anteriores para evaluar cualquier tendencia o cambio en las ventas.

Metodología:
Para llevar a cabo este análisis, se recopilaron y analizaron datos de ventas de productos durante el Día del Trabajador y en los días inmediatamente anteriores al evento. El objetivo es identificar cualquier variación en las ventas durante este período.

In [ ]:
dataframe_septiembre = dataframes_copia["Septiembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_septiembre= dataframe_septiembre.reset_index(inplace=False)
dataframe_septiembre

,Dia,Cantidad Pedida
0,1,447
1,2,452
2,3,433
3,4,444
4,5,406
5,6,433
6,7,415
7,8,425
8,9,442
9,10,443


Promedio de ventas en el mes de septiembre

In [ ]:
promedio_septiembre = dataframe_septiembre["Cantidad Pedida"].mean()
promedio_septiembre

437.3

Ventas Realizadas el dia del trabajador

In [ ]:
dataframe_septiemnbre_fest = dataframes_copia["Septiembre"].loc[dataframes_copia["Septiembre"]["Dia"] ==14].groupby('Dia')['Cantidad Pedida'].sum().reset_index()
dataframe_septiemnbre_fest

,Dia,Cantidad Pedida
0,14,452


Calculo del desvio standar

In [ ]:
desviacion_estandar_sept = dataframe_septiembre['Cantidad Pedida'].std()

print(f"La desviación estándar de 'Cantidad Pedida' es: {desviacion_estandar_sept}")


La desviación estándar de 'Cantidad Pedida' es: 17.958092979163347


La mayoría de los valores en la columna 'Cantidad Pedida' están a una distancia de aproximadamente 17.96 unidades de la unidad en la que medimos 'Cantidad Pedida' de la media.


Los 5 dias con mas ventas en el mes de Septiembre

In [ ]:
top_5_ventas_septiembre = dataframe_septiembre.nlargest(5, 'Cantidad Pedida')
print(top_5_ventas_septiembre)

    Dia  Cantidad Pedida
26   27              472
18   19              468
14   15              462
28   29              457
10   11              456


In [ ]:
# Dia del trabajador

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Datos para el mes de septiembre
mes = "Septiembre"
dias = list(range(1, 31))  # Lista de días del mes

fig = make_subplots(rows=1, cols=1, subplot_titles=[f"Productos por día en {mes}"])

dataframe_dia_trabaj = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_dia_trabaj = dataframe_dia_trabaj.reset_index(inplace=False)

# Colores de las barras, con rojo para el día 2 de septiembre
colores = ['red' if dia == 2 else 'blue' for dia in dataframe_dia_trabaj['Dia']]

# Agregar un gráfico de barras para septiembre con colores personalizados
bar_trace1 = go.Bar(
    x=dataframe_dia_trabaj['Dia'],
    y=dataframe_dia_trabaj['Cantidad Pedida'],
    name=f'{mes}',
    marker=dict(color=colores)  # Establecer colores personalizados
)

fig.update_xaxes(title_text="Días")
fig.update_yaxes(title_text="Cantidad")
fig.update_yaxes(range=[0, dataframe_dia['Cantidad Pedida'].max()])  # Ajustar el rango del eje Y

fig.add_trace(bar_trace1)

# Configurar el título
fig.update_layout(
    title=f'Productos por día en {mes}',
    width=800,  # Ancho en píxeles
)

fig.show(renderer="colab")



Se observa un leve aumento de ventas en el Día del Trabajador (Labor Day) en Estados Unidos, puedes realizar un análisis similar al que hicimos para otros eventos festivos.

Las ventas realizadas en el dia del trabajador se encuentran por encima del promedio de ventas realizadas en el mes de Septiembre.

Informe de Análisis de Ventas durante el Día del Trabajador - Septiembre

Introducción:
El presente informe tiene como objetivo analizar el impacto de un evento festivo en las ventas, específicamente en el Día del Trabajador (Labor Day) en Estados Unidos, que se celebra el primer lunes de septiembre. En este análisis, se examinan las ventas en el Día del Trabajador y se comparan con los días anteriores para evaluar cualquier tendencia o cambio en las ventas.

Metodología:
Para llevar a cabo este análisis, se recopilaron y analizaron datos de ventas de productos durante el Día del Trabajador y en los días inmediatamente anteriores al evento. El objetivo es identificar cualquier variación en las ventas durante este período.

Resultados:
Durante el análisis de ventas en el Día del Trabajador y los días previos, se observaron los siguientes hallazgos clave:

Leve Aumento de Ventas en el Día del Trabajador:
En el Día del Trabajador (primer lunes de septiembre), se registró un leve aumento en las ventas en comparación con los días anteriores.
Este aumento en las ventas sugiere que el Día del Trabajador podría influir en el comportamiento de compra de los consumidores, posiblemente debido a promociones o actividades relacionadas con el feriado.
Conclusiones:
Basado en el análisis de ventas durante el Día del Trabajador, se puede concluir que este evento festivo tiene un leve impacto positivo en las ventas. El aumento en las ventas en el Día del Trabajador indica que los consumidores podrían estar más activos en sus compras debido a las festividades y promociones asociadas con este día.



#### Dia de Colón

Introducción:
El presente informe tiene como objetivo analizar el impacto de un evento festivo en las ventas, específicamente en el Día de Colón (Columbus Day) en Estados Unidos, que se celebra el segundo lunes de octubre. En este análisis, se examinan las ventas en el Día de Colón y se comparan con los días anteriores y posteriores al evento para evaluar cualquier tendencia o cambio significativo en el comportamiento de las ventas.

Metodología:
Para llevar a cabo este análisis, se recopilaron y analizaron datos de ventas de productos durante el Día de Colón, así como en los dos días anteriores y los dos días posteriores al evento. El objetivo es identificar cualquier variación en las ventas durante este período.

In [ ]:
dataframe_octubre = dataframes_copia["Octubre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_octubre= dataframe_octubre.reset_index(inplace=False)
dataframe_octubre

,Dia,Cantidad Pedida
0,1,741
1,2,767
2,3,703
3,4,722
4,5,766
5,6,716
6,7,751
7,8,721
8,9,680
9,10,756


Promedio de ventas en octubre

In [ ]:
promedio_octubre = dataframe_octubre["Cantidad Pedida"].mean()
promedio_octubre

732.4193548387096

Ventas realizadas en el dia festivo de octubre

In [ ]:
dataframe_octubre_fest = dataframes_copia["Octubre"].loc[dataframes_copia["Octubre"]["Dia"] ==14].groupby('Dia')['Cantidad Pedida'].sum().reset_index()
dataframe_octubre_fest



,Dia,Cantidad Pedida
0,14,695


Calculo del Desvio standar

In [ ]:
desviacion_estandar_oct = dataframe_octubre['Cantidad Pedida'].std()

print(f"La desviación estándar de 'Cantidad Pedida' es: {desviacion_estandar_oct}")


La desviación estándar de 'Cantidad Pedida' es: 29.8460206097322


La mayoría de los valores en la columna 'Cantidad Pedida' están a una distancia de aproximadamente 29.845 unidades de la unidad en la que medimos 'Cantidad Pedida' de la media.


Top de los 5 dias con mayores cantidades pedidas

In [ ]:
top_5_ventas_colon = dataframe_dia_ind.nlargest(5, 'Cantidad Pedida')
print(top_5_ventas_colon)

    Dia  Cantidad Pedida
14   15              573
25   26              557
17   18              554
21   22              546
8     9              541


In [ ]:
#Dia de Colon
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Datos para el mes de octubre
mes = "Octubre"
dias = list(range(1, 32))  # Lista de días del mes

fig = make_subplots(rows=1, cols=1, subplot_titles=[f"Productos por día en {mes}"])

dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_dia = dataframe_dia.reset_index(inplace=False)

# Colores de las barras, con rojo para el día 14 de octubre
colores = ['red' if dia == 14 else 'blue' for dia in dataframe_dia['Dia']]

# Agregar un gráfico de barras para octubre con colores personalizados
bar_trace1 = go.Bar(
    x=dataframe_dia['Dia'],
    y=dataframe_dia['Cantidad Pedida'],
    name=f'{mes}',
    marker=dict(color=colores)  # Establecer colores personalizados
)

fig.update_xaxes(title_text="Días")
fig.update_yaxes(title_text="Cantidad")
fig.update_yaxes(range=[0, dataframe_dia['Cantidad Pedida'].max()])  # Ajustar el rango del eje Y

fig.add_trace(bar_trace1)

# Configurar el título
fig.update_layout(
    title=f'Productos por día en {mes}',
    width=800,  # Ancho en píxeles
)

fig.show(renderer="colab")


Resultados:
Durante el análisis de ventas en el Día de Colón y los días circundantes, se observaron los siguientes hallazgos clave:

Disminución de Ventas en el Día de Colón:

En el Día de Colón (segundo lunes de octubre), se registró una disminución significativa en las ventas en comparación con los días anteriores.
Esta disminución en las ventas sugiere que el Día de Colón tiene un impacto notable en el comportamiento de compra de los consumidores.

Las ventas realizadas en el Día de Colón, se encuentran notablemente por debajo del promedio de ventas realizadas en Octubre.

Tendencia a la Normalización Posterior al Evento:

Después del Día de Colón, en los dos días posteriores, se observó una tendencia a la normalización en las ventas.
Los consumidores parecen volver gradualmente a su actividad de compra habitual después de la celebración del evento.
Conclusiones:
Basado en el análisis de ventas durante el Día de Colón, se puede concluir que este evento festivo tiene un impacto significativo en las ventas. La marcada disminución de las ventas en el Día de Colón indica que los consumidores pueden estar dedicando su tiempo a actividades relacionadas con el feriado y, por lo tanto, reduciendo su actividad de compra.

Es importante destacar que las ventas tienden a recuperarse en los días posteriores al Día de Colón, lo que sugiere que los consumidores eventualmente vuelven a su comportamiento de compra normal. Sin embargo, este análisis destaca la importancia de tener en cuenta los eventos festivos al planificar estrategias de ventas y gestión de inventario.



#### Día de los Veteranos y Día de Accion de Gracias

Día de los Veteranos - Noviembre:

En el análisis de ventas durante el Día de los Veteranos (11 de noviembre), se observó una disminución muy significativa en las ventas en comparación con los días anteriores y posteriores.
Esta disminución en las ventas sugiere que el Día de los Veteranos tiene un impacto notable en el comportamiento de compra de los consumidores, posiblemente debido a la observancia y actividades relacionadas con el feriado.
Día de Acción de Gracias - Noviembre:

Durante el análisis de ventas en el Día de Acción de Gracias (cuarto jueves de noviembre), se registró una disminución muy significativa en las ventas en comparación con los días anteriores.
Esta disminución en las ventas podría atribuirse al hecho de que los consumidores están ocupados con celebraciones y actividades familiares durante este feriado.

In [ ]:
dataframe_noviembre = dataframes_copia["Noviembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_noviembre= dataframe_noviembre.reset_index(inplace=False)
dataframe_noviembre

,Dia,Cantidad Pedida
0,1,714
1,2,671
2,3,604
3,4,700
4,5,643
5,6,644
6,7,699
7,8,710
8,9,667
9,10,636


Promedio ventas mes de noviembre
promedio

In [ ]:
promedio_noviembre = dataframe_noviembre["Cantidad Pedida"].mean()
promedio_noviembre

660.2666666666667

Calculo del Desvio Standar

In [ ]:
desviacion_estandar_nov = dataframe_noviembre['Cantidad Pedida'].std()

print(f"La desviación estándar de 'Cantidad Pedida' es: {desviacion_estandar_nov}")


La desviación estándar de 'Cantidad Pedida' es: 33.8168834932405


La mayoría de los valores en la columna 'Cantidad Pedida' están a una distancia de aproximadamente 33.81 unidades de la unidad en la que medimos 'Cantidad Pedida' de la media.


Ventas en los dias festivos correspondiente al mes de Noviembre

In [ ]:
dataframe_noviembre_fest = dataframes_copia["Noviembre"].loc[dataframes_copia["Noviembre"]["Dia"].isin([11, 28])].groupby('Dia')['Cantidad Pedida'].sum().reset_index()
dataframe_noviembre_fest

,Dia,Cantidad Pedida
0,11,622
1,28,594


Calculo los 5 dias con mayores ventas

In [ ]:
top_5_ventas_noviembre = dataframe_noviembre.nlargest(5, 'Cantidad Pedida')
print(top_5_ventas_noviembre)

    Dia  Cantidad Pedida
20   21              718
0     1              714
7     8              710
3     4              700
23   24              700


In [ ]:
#Día de los Veteranos (Veterans Day) - 11 de noviembre  color rojo
# Día de Acción de Gracias (Thanksgiving Day) - en el año 2019 cayo dia 28

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Datos para el mes de noviembre
mes = "Noviembre"
dias = list(range(1, 31))  # Lista de días del mes

fig = make_subplots(rows=1, cols=1, subplot_titles=[f"Productos por día en {mes}"])

dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_dia = dataframe_dia.reset_index(inplace=False)

# Colores de las barras, con rojo para el Día de los Veteranos (11 de noviembre) y violeta para el Día de Acción de Gracias (28 de noviembre)
colores = ['red' if dia == 11 else 'purple' if dia == 28 else 'blue' for dia in dataframe_dia['Dia']]

# Agregar un gráfico de barras para noviembre con colores personalizados
bar_trace1 = go.Bar(
    x=dataframe_dia['Dia'],
    y=dataframe_dia['Cantidad Pedida'],
    name=f'{mes}',
    marker=dict(color=colores),
    showlegend=True,  # Mostrar la leyenda
    legendgroup="Días Especiales",  # Agrupar los colores en la leyenda
)

fig.update_xaxes(title_text="Días")
fig.update_yaxes(title_text="Cantidad")
fig.update_yaxes(range=[0, dataframe_dia['Cantidad Pedida'].max()])  # Ajustar el rango del eje Y

fig.add_trace(bar_trace1)

# Configurar el título y la leyenda de colores
fig.update_layout(
    title=f'Productos por día en {mes}',
    width=800,  # Ancho en píxeles
    legend=dict(title="Días Especiales", tracegroupgap=0),  # Título de la leyenda
)

fig.show(renderer="colab")


Conclusiones:
Basado en el análisis de ventas para el Día de los Veteranos y el Día de Acción de Gracias, se puede concluir que ambos eventos festivos tienen un impacto muy significativo en las ventas. La disminución en las ventas durante estos días sugiere que los consumidores pueden estar más enfocados en las festividades y actividades relacionadas con los feriados que en las compras habituales.

Este análisis destaca la importancia de considerar estos eventos festivos al planificar estrategias de ventas y gestión de inventario. Puede ser beneficioso adaptar las estrategias comerciales para atender las expectativas cambiantes de los consumidores durante estos días festivos.

La cantidad de ventas en ambos dias festivos en noviembre, se encuentra debajo del promedio de ventas del mes de noviembre

#### Navidad y ultimo día del año

Navidad - Diciembre:

Durante el análisis de ventas para la temporada navideña, se observó un patrón claro de fuerte aumento en las ventas en los dos días previos a Navidad (24 y 23 de diciembre).
Estos incrementos en las ventas pueden atribuirse a las compras de último momento y la preparación para las festividades navideñas.
Sin embargo, el día festivo de Navidad (25 de diciembre) mostró una disminución significativa en las ventas, lo que sugiere que la mayoría de las compras se realizaron antes de la fecha festiva.
Último Día del Año - Diciembre:

En el análisis de ventas para el último día del año (31 de diciembre), se observó una disminución considerable en las ventas en comparación con los dos días previos (30 y 29 de diciembre).
Esta disminución en las ventas el 31 de diciembre podría atribuirse a que los consumidores están ocupados preparándose para las celebraciones de Año Nuevo y no están tan enfocados en las compras.

In [ ]:
dataframe_diciembre = dataframes_copia["Diciembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_diciembre= dataframe_diciembre.reset_index(inplace=False)
dataframe_diciembre

,Dia,Cantidad Pedida
0,1,884
1,2,914
2,3,832
3,4,879
4,5,905
5,6,910
6,7,843
7,8,870
8,9,856
9,10,897


Filtro el 25 y el 31 de Diciembre que serian dias festivos en diciembre

In [ ]:
dataframe_diciembre_fest = dataframes_copia["Diciembre"].loc[dataframes_copia["Diciembre"]["Dia"].isin([25, 31])].groupby('Dia')['Cantidad Pedida'].sum().reset_index()
dataframe_diciembre_fest

,Dia,Cantidad Pedida
0,25,902
1,31,886


Promedio de ventas del mes

In [ ]:
promedio_dic = dataframe_diciembre["Cantidad Pedida"].mean()
promedio_dic

907.1290322580645

Calculo del Desvio standar

In [ ]:
desviacion_estandar_dic = dataframe_diciembre['Cantidad Pedida'].std()

print(f"La desviación estándar de 'Cantidad Pedida' es: {desviacion_estandar_dic}")


La desviación estándar de 'Cantidad Pedida' es: 42.388474797979285


La mayoría de los valores en la columna 'Cantidad Pedida' están a una distancia de aproximadamente 42.39 unidades de la unidad en la que medimos 'Cantidad Pedida' de la media.


Calculo los5 dias de diciembre con mayores ventas

In [ ]:
top_5_ventas_diciembre = dataframe_diciembre.nlargest(5, 'Cantidad Pedida')
print(top_5_ventas_diciembre)

    Dia  Cantidad Pedida
16   17             1044
22   23              988
12   13              957
28   29              954
18   19              942


In [ ]:
# Navidad 25 de diciembre y ultimo dia del año 31 de diciembre

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Datos para el mes de diciembre
mes = "Diciembre"
dias = list(range(1, 32))  # Lista de días del mes

fig = make_subplots(rows=1, cols=1, subplot_titles=[f"Productos por día en {mes}"])

dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_dia = dataframe_dia.reset_index(inplace=False)

# Colores de las barras, con rojo para el 25 de diciembre y violeta para el 31 de diciembre
colores = ['red' if dia == 25 else 'purple' if dia == 31 else 'blue' for dia in dataframe_dia['Dia']]

# Agregar un gráfico de barras para diciembre con colores personalizados
bar_trace1 = go.Bar(
    x=dataframe_dia['Dia'],
    y=dataframe_dia['Cantidad Pedida'],
    name=f'{mes}',
    marker=dict(color=colores),
    showlegend=True,  # Mostrar la leyenda
    legendgroup="Días Especiales",  # Agrupar los colores en la leyenda
)

fig.update_xaxes(title_text="Días")
fig.update_yaxes(title_text="Cantidad")
fig.update_yaxes(range=[0, dataframe_dia['Cantidad Pedida'].max()])  # Ajustar el rango del eje Y

fig.add_trace(bar_trace1)

# Configurar el título y la leyenda de colores
fig.update_layout(
    title=f'Productos por día en {mes}',
    width=800,  # Ancho en píxeles
    legend=dict(title="Días Especiales", tracegroupgap=0),  # Título de la leyenda
)

fig.show(renderer="colab")


Conclusiones:
Basado en el análisis de ventas para Navidad y el último día del año, se puede concluir lo siguiente:

Para Navidad, se observa un patrón caracterizado por un fuerte incremento en las ventas en los dos días previos, lo que indica una alta demanda de compras de último momento. Sin embargo, el día festivo de Navidad experimenta una disminución significativa en las ventas, lo que sugiere que la mayoría de las compras se realizan antes de esa fecha.
En cuanto al 31 de diciembre, las ventas disminuyen considerablemente en comparación con los dos días previos, lo que podría relacionarse con la prioridad de los consumidores en las celebraciones de Año Nuevo en lugar de las compras.
Ambos dias poseen una cantidad de ventas, que se encuentran por debajo del promedio de ventas realizadas en el mes


## En sintesis

Aumento o Disminución Significativa en las Ventas en Días Cercanos a Eventos Especiales:

**Día de la Independencia (Independence Day)** - 4 de julio:

En cuanto al 4 de julio, no se observaron picos significativos en las ventas ni aumentos ni disminuciones notables.
Las ventas se mantuvieron relativamente constantes en los días cercanos a esta fecha.
Parece que el Día de la Independencia no tiene un impacto significativo en el comportamiento de las ventas, y las ventas se mantienen en niveles similares a los días anteriores.

**Día de los Veteranos (Veterans Day)** - 11 de noviembre:

Se observó una disminución significativa en las ventas en el Día de los Veteranos.
Este evento festivo parece tener un impacto negativo en las ventas, posiblemente debido a la observancia y actividades relacionadas con el feriado.

**Día de Acción de Gracias (Thanksgiving Day)** - Cuarto jueves de noviembre:

Se registró una disminución muy significativa en las ventas durante el Día de Acción de Gracias.
Los consumidores parecen estar más enfocados en las celebraciones y actividades familiares que en las compras.

**Navidad (Christmas Day) - 25 de diciembre:**

Hubo un fuerte aumento en las ventas en los días previos a Navidad (24 y 23 de diciembre).
Sin embargo, el día festivo de Navidad experimentó una disminución significativa en las ventas, lo que sugiere que la mayoría de las compras se realizan antes de esa fecha.

**Último Día del Año** (31 de diciembre):

Las ventas disminuyeron considerablemente el 31 de diciembre en comparación con los dos días previos (30 y 29 de diciembre).
Esto podría deberse a que los consumidores están ocupados preparándose para las celebraciones de Año Nuevo y no están tan enfocados en las compras.

**Día del Trabajador (Labor Day)** - Primer lunes de septiembre:

Se observó un leve aumento de ventas en el Día del Trabajador.
Este evento festivo parece tener un impacto positivo en las ventas, posiblemente debido a promociones o actividades relacionadas con el feriado.

**Día de Colón (Columbus Day) - Segundo lunes de octubre**:

También se observó una disminución en las ventas durante el Día de Colón.
Esto sugiere que este evento festivo tiene un impacto negativo en el comportamiento de compra de los consumidores.
Eventos Específicos con Impacto Notable:

**El Día de Acción de Gracias y el Día de los Veteranos** han tenido un impacto notable en el comportamiento de las ventas, con disminuciones significativas en las ventas durante estos eventos festivos.
Para Navidad, se observa un fuerte aumento de ventas en los días previos, pero una disminución significativa el día festivo en sí.

El Día del Trabajador muestra un leve aumento de ventas, indicando un impacto positivo en las ventas.

El Día de Colón también ha mostrado una disminución en las ventas durante el evento festivo.
En resumen, varios eventos especiales tienen un impacto significativo en las ventas, en su mayoria las ventas en los dias festivos disminuyeron notablemente.

Conlusion Final:
De acuerdo a los datos obtenidos, podemos determinar que las ventas de productos en días festivos se ve disminuida, por lo que pareceria indicar  que los clientes, se encuentran a disfrutar ese día.

Segundo inciso
Informe sobre el Incremento de Ventas en Octubre

Introducción:

En este informe, se analizará el notable incremento en las ventas totales durante el mes de octubre en comparación con el mes de septiembre. Examinaremos cómo factores externos del mercado y la elasticidad de la demanda de las baterías pueden haber influido en este fenómeno.

Análisis de Ventas Mensuales:

La gráfica de ventas mensuales muestra claramente que las ventas totales en octubre superaron significativamente las ventas de septiembre en nuestro negocio. Este aumento inesperado de las ventas ha generado interés y curiosidad.

Posibles Factores Externos del Mercado:

Uno de los factores externos que podrían haber contribuido a este incremento podría estar relacionado con una noticia publicada en el diario The New York Times en español el 4 de septiembre de 2019 (https://www.nytimes.com/es/2019/09/04/espanol/mundo/corea-del-norte-arsenal-trump.html). El titular de la noticia hace referencia a los desarrollos en Corea del Norte con respecto a su arsenal nuclear y las acciones del entonces presidente de los Estados Unidos, Donald Trump. Aunque el titular menciona el refuerzo del arsenal norcoreano y la desestimación de pruebas de misiles, es importante destacar que en el contenido de la noticia se hace referencia a la preocupación de ciudadanos estadounidenses que podrían verse afectados por estos acontecimientos.

Impacto del Temor a una Guerra:
El temor a una posible guerra o conflicto internacional puede haber influido en las decisiones de compra de los consumidores. Este temor podría haber llevado a las personas a adquirir ciertos bienes y recursos que consideran esenciales en caso de circunstancias extraordinarias. Este comportamiento es común durante períodos de incertidumbre y tensión internacional, donde los consumidores tienden a acumular productos esenciales como baterías, alimentos enlatados, agua y otros suministros.

Elasticidad de la Demanda de las Baterías:
Además de los factores mencionados anteriormente, es importante destacar que la elasticidad de la demanda de las baterías también puede haber influido en el aumento de ventas. En un mundo cada vez más digital y tecnológico, surgen constantemente nuevos dispositivos electrónicos que utilizan baterías como fuente de energía. La creciente necesidad de baterías para alimentar estos dispositivos ha creado una demanda sostenida para este producto.

Conclusión:
El aumento sustancial de ventas en octubre en comparación con septiembre podría atribuirse a una combinación de factores. La incertidumbre geopolítica relacionada con la noticia de Corea del Norte puede haber impulsado la compra de bienes esenciales. Además, la elasticidad de la demanda de las baterías en un mundo tecnológico en constante evolución ha contribuido a un aumento constante en la demanda de estos productos.

